In [ ]:
import pandas as pd
from util.readme_parser import ReadmeSectionParser
from e2e_system.reproder import HierarchicalReproder, _ground_truth

import os
from tqdm.auto import tqdm
import json
from datetime import datetime
import shutil
import glob
from util.file_helper import get_readme_with_repo_name

import warnings
warnings.filterwarnings("ignore")


In [ ]:
keys = ["header", "parent_header", "content"]
model = "hierarchical"
model_paths = glob.glob(f"model/{model}/**")
neurips_df = pd.read_csv(
    'data/paperswithcode/neurips_checklist_manuel-100.csv')
neurips_df["score"] = neurips_df.select_dtypes(bool).sum(axis=1)


In [ ]:
def evaluate(reproder: HierarchicalReproder, m_df, type=1):
    df = m_df.copy()
    for repo in (pbar := tqdm(neurips_df.url.values, desc="Evaluating")):
        pbar.set_postfix_str(f"Evaluating {repo}")
        readme = get_readme_with_repo_name("neurips", repo)
        sections = ReadmeSectionParser(
            repo, readme).parse_sections(group_by_parent=False)
        record = df[df["url"] == repo]
        if len(sections) == 0:
            df.loc[record.index, "reprod_score"] = 0
            continue
        readme_text = ReadmeSectionParser.merge_sections(
            pd.DataFrame(sections), with_newline=True, keys=keys)
        logits = reproder.evaluate(readme_text)
        reprod_score = reproder.calculate_reproducibility(logits, type=type)
        record = df[df["url"] == repo]
        df.loc[record.index, "reprod_score"] = reprod_score
    return df


In [ ]:
for model in (pbar := tqdm(model_paths)):
    pbar.set_postfix_str(f"Evaluating {model}")
    reproder = HierarchicalReproder(model, keys)
    direct_classification = evaluate(reproder, neurips_df, type=1)
    weighted_score = evaluate(reproder, neurips_df, type=2)
    direct_coeff = evaluate(reproder, neurips_df, type=3)

    save_main_dir = f'data/paperswithcode/evaluated/hierarchical/{model}/'
    save_dir = os.path.join(save_main_dir, model.split("\\")[-1])
    shutil.rmtree(save_dir, ignore_errors=True)
    os.makedirs(save_dir, exist_ok=True)
    info = {
        "model_dir": model,
        "keys": keys,
        "time": datetime.now().strftime("%Y%m%d-%H%M%S")
    }
    # open(os.path.join(save_dir, "info.json"), 'w').write(
    #     json.dumps(info, indent=4))

    # direct_classification.to_csv(
    #     f'{save_dir}/direct_classification.csv', index=False)
    # weighted_score.to_csv(
    #     f'{save_dir}/weighted_score.csv', index=False)
    # direct_coeff.to_csv(
    #     f'{save_dir}/direct_coeff.csv', index=False)
